1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normal
distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
   * $x_1$ is distributed as $N(0,1)$
   * $x_2$ is distributed as $x_1+N(0,3)$
   * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset
was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of
the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots.
Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.

In [32]:
import numpy as np
import numpy.linalg as la

#
sample_size = 1000
x1 = np.random.normal(0.0, 1.0, size=sample_size)
x2 = x1 + np.random.normal(0.0, 3.0, size=sample_size)
x3 = 2 * x1 + x2
x = np.vstack((x1, x2, x3))
print(f"{x = }")
#
x_cov = np.cov(x)
print(f"{x_cov = }")
eigenvalues, eigenvectors = la.eig(x_cov)
print(f"{eigenvalues = }")
print(f"{eigenvectors = }")
#
u, s, vh = la.svd(x_cov)
print(f"{vh = }")

x = array([[ 0.17813162, -0.06391786, -0.77585112, ...,  0.46680059,
        -0.348629  , -1.02536149],
       [ 1.68861556,  2.59775392, -6.97506355, ...,  2.20629963,
        -1.0841628 ,  2.98755478],
       [ 2.04487881,  2.4699182 , -8.52676579, ...,  3.13990081,
        -1.78142079,  0.9368318 ]])
x_cov = array([[ 1.01430372,  0.89767707,  2.92628451],
       [ 0.89767707,  8.9165198 , 10.71187394],
       [ 2.92628451, 10.71187394, 16.56444296]])
eigenvalues = array([2.44757398e+01, 2.70687696e-16, 2.01952672e+00])
eigenvectors = array([[-0.12325102, -0.81649658,  0.56404124],
       [-0.5675556 , -0.40824829, -0.71499229],
       [-0.81405763,  0.40824829,  0.41309019]])
vh = array([[-0.12325102, -0.5675556 , -0.81405763],
       [ 0.56404124, -0.71499229,  0.41309019],
       [ 0.81649658,  0.40824829, -0.40824829]])


2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise.
Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar
deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record
the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$,
the angles in spherical coordinates.

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the X axis,
assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate
the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding
the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [ ]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 